In [1]:
!pip install streamlit -q
!pip install pyngrok
!pip install finance-datareader
!pip install pykrx
!pip install cryptocmd
!pip install mplfinance

     |████████████████████████████████| 9.2 MB 5.3 MB/s 
     |████████████████████████████████| 164 kB 48.7 MB/s 
     |████████████████████████████████| 237 kB 39.4 MB/s 
     |████████████████████████████████| 182 kB 54.5 MB/s 
     |████████████████████████████████| 4.7 MB 51.1 MB/s 
     |████████████████████████████████| 78 kB 6.0 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 51 kB 5.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 5.7 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=68042d556719272e8288e8d3c7a6474775ca7e496855c3da3df7e0a579dee466
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Lo

In [2]:
from pyngrok import ngrok

ngrok.set_auth_token('token')

### 주식 분석 웹페이지 (비트코인 추가)

In [ ]:
# Referenced these links
# https://ai-creator.tistory.com/m/307
# https://streamlit.io/components
# https://velog.io/@anjaekk/Python-Pandas-Streamlit%EB%A1%9C-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EC%8B%9C%EA%B0%81%ED%99%94-%ED%95%98%EA%B8%B0
# https://mokeya.tistory.com/72
# https://colab.research.google.com/github/teddylee777/machine-learning/blob/master/04-TensorFlow2.0/01-%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90-%EC%A3%BC%EA%B0%80%EC%98%88%EC%B8%A1/02-LSTM-stock-forecasting-with-LSTM-financedatareader.ipynb#scrollTo=q80PTU2MoNSn
# https://sjblog1.tistory.com/56
# https://khw11044.github.io/blog/library/2021-01-18-streamlit/
# https://coinmarketcap.com
# https://youngwonhan-family.tistory.com/32

In [ ]:
import mplfinance as mpf
import pandas_datareader as web

df = web.naver.NaverDailyReader('005930', start='20201201' , end='20210119').read()
df.reset_index(inplace=True)
df

In [17]:
%%writefile app.py

from pyngrok import ngrok
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mlp
import requests
from bs4 import BeautifulSoup as bs
import FinanceDataReader as fdr
import time
from pykrx import stock
import streamlit as st
from cryptocmd import CmcScraper
import pandas_datareader as pdr
import plotly.express as px
from datetime import datetime, timedelta
import mplfinance as mpf

def get_stock_code(stock_type):
    stock_list = fdr.StockListing(stock_type)
    stock_list = stock_list[['Name', 'Code']]
    stock_list = stock_list.rename(columns={'Name': 'company', 'Code': 'code'})

    return stock_list

def get_stock(code, start, end):
    df = pdr.naver.NaverDailyReader(code, start=start , end=end).read()
    df.reset_index(inplace=True)
    return df

def except_coin(stock_type):
    stock_code = get_stock_code(stock_type)

    option = st.sidebar.selectbox('알고 싶은 회사를 선택해주세요.', (stock_code['company']))
    start_date = st.sidebar.date_input('시작 날짜를 선택해주세요.', datetime.today() - timedelta(7))
    end_date = st.sidebar.date_input('마칠 날짜를 선택해주세요.', datetime.today())

    code = stock_code[stock_code.company==option].code.values[0].strip()
    # start_date = start_date.strftime('%d%m%y')
    # end_date = end_date.strftime('%d%m%y')

    df = get_stock(code, start_date, end_date)
    df[['Open', 'High', 'Low', 'Close']] = df[['Open', 'High', 'Low', 'Close']].astype(dtype='int64',errors='ignore')
    df['Volume']= df['Volume'].astype(dtype='int64',errors='ignore')

    fig_close = px.line(df, x='Date', y=['Open', 'High', 'Low', 'Close'], title='가격')
    fig_volume = px.line(df, x='Date', y=['Volume'], title='거래량')

    st.plotly_chart(fig_close)
    st.plotly_chart(fig_volume)

st.title('주분싸 WEB app')
st.sidebar.header('Menu')

stock_type = st.sidebar.selectbox('거래소를 선택해주세요.', ['KRX', 'CoinMarketCap(임시)'])

if stock_type == 'KRX':
    except_coin(stock_type)

elif stock_type == 'CoinMarketCap(임시)':
    coin = st.sidebar.selectbox('코인을 선택해주세요.', ['BTC', 'ETH', 'USDT'])

    start_date = st.sidebar.date_input('시작 날짜를 선택해주세요.', datetime.today() - timedelta(8))
    end_date = st.sidebar.date_input('마칠 날짜를 선택해주세요.', datetime.today() - timedelta(1))

    scraper = CmcScraper(coin, start_date.strftime('%d-%m-%Y'), end_date.strftime('%d-%m-%Y'))

    df = scraper.get_dataframe()

    fig_close = px.line(df, x='Date', y=['Open', 'High', 'Low', 'Close'], title='가격')
    fig_volume = px.line(df, x='Date', y=['Volume'], title='거래량')

    st.plotly_chart(fig_close)
    st.plotly_chart(fig_volume)

Overwriting app.py


In [8]:
!nohup streamlit run app.py --server.port 80 &

url = ngrok.connect(port='80')
url

nohup: appending output to 'nohup.out'


<NgrokTunnel: "http://51b6-34-74-156-30.ngrok.io" -> "http://localhost:80">

In [18]:
ngrok.kill()